 ## Transformer Model

![Transformer Model](transformer1.png)

Bu Notebookta sizlere bir Transformer modeli kodlayacağım. Transformer mimarisi, Ashish Vaswani ve diğerleri tarafından yazılan [Attention is all you need](https://arxiv.org/abs/1706.03762) makalesine dayanmaktadır. Model, [PyTorch](https://pytorch.org/) kullanılarak implement edilmiştir.

  ### 1. Kütüphaneleri İçe Aktarma

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import time
import math
# Eğer GPU kullanılabiliyorsa (Gerçi bu modelde GPU kullanmak bir seçenekten ziyade ne yazık ki bir zorunluluk :D)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


 ### 2. Hiperparametreleri Tanımlama

In [ ]:
d_model= 256 # Embed size olarak da bilinir: token embedding vektörünün boyutu
nhead = 4 # Multi Head Attention katmanındaki head sayısı
num_encoder_layers = 1 # Encoderdaki EncoderStack sayısı
num_decoder_layers = 1 # Decoderdaki DecoderStack sayısı
forward_expansion= 4 # Transformer Encoder/Decoder içindeki ileri beslemeli ağdaki nöronların artış oranı
learning_rate = 3e-4 # Adam optimizer'ının learning rate değeri
block_size = 128 # Girdi dizisinin (maksimum) uzunluğu
vocab_size = 30000 # Kelime haznesinin boyutu (vocab_size) (Tokenizer tarafından bilinen token sayısı)
dropout = 0.25 # Dropout katmanlarının atma oranı


 ### 3. Transformer Bloklarını Oluşturma

 #### 3.1. Positional Encoding

 Bu bölümde Positional Encoding (pozisyonel kodlama), makalede açıklandığı gibi kullanacağız.

 Bunun için aşağıdaki formülü kullanacağız:


 $$PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})$$
 $$PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})$$

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        position = torch.arange(self.max_sequence_length, device=device, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2, device=device, dtype=torch.float32) * (-math.log(10000.0) / self.d_model))
        pe = torch.zeros(self.max_sequence_length, self.d_model, device=device, dtype=torch.float32)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe


 #### 3.2. Embedding Bloğu

 Bu bölümde Embedding bloğunu oluşturacağız. Bu blok, girdi verilerini bir vektör temsiline dönüştürmek için kullanılacak.

 Bu blok temel olarak 2 bölümden oluşuyor:
 1. Token Embedding
 2. Positional Encoding

 Positional Encoding bloğunu yukarıda ayrı bir sınıf olarak tanımladık.

 Bu nedenle şimdi 


In [ ]:
class EmbeddingLayer(nn.Module):
    def __init__(self, d_model, vocab_size, block_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, block_size)
    def forward(self, x):
        out = self.token_embedding(x) + self.positional_encoding()
        return  out


 #### 3.3. Multi Head Attention

 Bu bölümde Multi Head Attention katmanını uygulayacağız. Çoklu kafa dikkat katmanı, $h$ adet dikkat başlığına sahiptir. Her dikkat başlığının ayrı bir sorgu, anahtar ve değer matrisi vardır.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        # d_model'in num_heads'e bölünebilir olup olmadığını kontrol ediyoruz
        assert d_model % num_heads == 0, "d_model num_heads'e bölünebilmelidir"
        
        # Her başlığa d_model'in num_heads'e bölünmüş boyutunu atamak için d_model'i num_heads'e bölüyoruz. Bu hem d_k hem de d_v için geçerlidir
        self.d_qkv = d_model // num_heads

        # Sorguları, anahtarları ve değerleri projekte etmek için nn.Linear kullanıyoruz. Aynı lineer projeksiyonu tüm başlıklar için kullanıyoruz.
        # Bunun nedeni, sorguları, anahtarları ve değerleri projekte etmek için ayrı ayrı matrisler kullanmak yerine tek bir matris çarpımı kullanmamıza olanak tanımasıdır.
        # Bu, her biri için ayrı matrisler kullanmaktan daha verimlidir.
        self.W_keys = nn.Linear(d_model, d_model)
        self.W_queries = nn.Linear(d_model, d_model)
        self.W_values = nn.Linear(d_model, d_model)

        # Çıktıyı projekte etmek için tek bir lineer projeksiyon kullanıyoruz
        self.linear_proj = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout) 

    def forward(self, key_src, query_src, value_src, mask=None):
        
        # Girdi yığınının şeklini alıyoruz
        B,T,C = key_src.shape # (batch_size, seq_len, d_model)


        # Sorguları, anahtarları ve değerleri ilgili ağırlık matrislerini kullanarak projekte ediyoruz
        keys = self.W_keys(key_src) # (batch_size, seq_len, d_model)
        queries = self.W_queries(query_src) # (batch_size, seq_len, d_model)
        values = self.W_values(value_src) # (batch_size, seq_len, d_model)
        

        # Sorguları, anahtarları ve değerleri çoklu başlıklara bölmek için yeniden şekillendiriyoruz
        
        keys = keys.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)
        queries = queries.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)
        values = values.view(B,T,self.num_heads,self.d_qkv) # (batch_size, seq_len, num_heads, d_qkv)


        # Sorguları, anahtarları ve değerleri tensörün şeklini (batch_size, num_heads, seq_len, d_qkv) yapacak şekilde yer değiştiriyoruz

        keys = keys.transpose(1,2) # (batch_size, num_heads, seq_len, d_qkv)
        queries = queries.transpose(1,2) # (batch_size, num_heads, seq_len, d_qkv)
        values = values.transpose(1,2) # (batch_size, num_heads, seq_len, d_qkv)

        # Dikkat puanlarını hesaplıyoruz.
        atn_scr = queries @ keys.transpose(-2,-1) # (batch_size, num_heads, seq_len, seq_len)
        # Dikkat puanlarını ölçeklendiriyoruz ve maskeyi uyguluyoruz (varsa)
        scaled_atn_scr = atn_scr / self.d_qkv**-0.5
        if mask is not None:
            scaled_atn_scr = scaled_atn_scr.masked_fill(mask==0,float('-inf'))
        
        # Dikkat ağırlıklarını hesaplamak için softmax aktivasyonunu uyguluyoruz
        attention_weights = torch.softmax(scaled_atn_scr, dim=-1)
        attention_weights = self.dropout(attention_weights)  # Dropout uyguluyoruz
        # Son olarak, dikkat ağırlıklarını değerlerle çarpıyoruz
        out = attention_weights @ values
        out = out.transpose(1, 2)
        # Matrisi (batch_size, seq_len, d_model) şekline dönüştürmek için yeniden şekillendiriyoruz
        out = out.reshape(B, T, C)
        # Bir sonraki katmana beslemek için son bir lineer projeksiyon uyguluyoruz
        out = self.linear_proj(out)
        return out


 #### 3.4. İleri Beslemeli Sinir Ağı

 Bu bölümde makalede olduğu gibi FFN'yi uygulayacağız. Bu, ağın içinde 2 lineer katman ve bu katmanlar arasında ReLU aktivasyon fonksiyonunun olduğu anlamına gelir. Ağın giriş ve çıkış boyutu aynı kalır, ancak içeride boyutu bir katsayı ile çarpmak üzere artırırız, bu katsayıya "Forward Expansion" denir.

In [ ]:
class FeedForwardNet(nn.Module):
    def __init__(self, d_model, forward_expansion, dropout=0.1):  
        super(FeedForwardNet, self).__init__()
        # İlk lineer katmanın çıkış boyutu forward_expansion kez d_model (d_model*forward_expansion)
        self.fc1 = nn.Linear(d_model, d_model * forward_expansion)
        self.relu = nn.ReLU()
        # İkinci lineer katmanın giriş boyutu d_model * forward_expansion ve çıkışı sadece d_model'dir 
        # İleri beslemeli ağa giren girdinin boyutunu aynı tutabilmek ve artıklı bağlantı kullanabilmek için
        self.fc2 = nn.Linear(d_model * forward_expansion, d_model)

        self.dropout = nn.Dropout(dropout)  

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)  
        out = self.fc2(out)
        return out


 #### 3.5. Encoder Stack

 Bu bölümde Encoder Stack oluşturacağız. Encoder Stack 1 MHA ve 1 FFN bloğundan oluşur. Bu bloğun her alt birimini Layer Normalization ve Residual Connection takip eder.

In [ ]:
class EncoderStack(nn.Module):
    def __init__(self, d_model, num_heads, forward_expansion, dropout=0.1): 
        super().__init__()
        self.MHA = MultiHeadAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
        self.FFN = FeedForwardNet(d_model=d_model, forward_expansion=forward_expansion, dropout=dropout)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout)  # Dropout katmanı eklendi
        self.dropout2 = nn.Dropout(dropout)  # Dropout katmanı eklendi

    def forward(self, x):
        out = x + self.dropout1(self.MHA(x, x, x))  # Dropout uyguluyoruz + Residual connection
        norm_out = self.layer_norm1(out) # Layer Normalization uyguluyoruz
        out = norm_out + self.dropout2(self.FFN(norm_out))  # Dropout uyguluyoruz + Residual connection
        norm_out = self.layer_norm2(out) # Layer Normalization uyguluyoruz
        return norm_out


 #### 3.6. Encoder

 Bu bölümde Encoder bloğunu oluşturuyoruz. Encoder bloğu bir Embedding katmanı ve N adet Encoder Stack bloğundan (N = num_layers) oluşur.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, block_size, d_model, num_heads, forward_expansion, num_layers):
        super().__init__()
        self.block_size = block_size
        self.d_model = d_model
        self.embeding_layer = EmbeddingLayer(d_model, vocab_size, block_size)
        self.layers = nn.ModuleList([EncoderStack(d_model, num_heads, forward_expansion) for _ in range(num_layers)])
    
    def forward(self, x):
        x = self.embeding_layer(x)
        for layer in self.layers:
            x = layer(x)
        return x


 #### 3.7. Decoder Stack

 Bu bölümde Decoder Stack bloğunu oluşturacağız. Decoder Stack bloğu, 2 MHA (birisi maskelemeli dikkat, diğeri çapraz dikkat için) ve 1 FFN bloğundan oluşur. Bu bloğun da her alt birimini Layer Normalization ve Residual Connection takip eder.

In [ ]:
class DecoderStack(nn.Module):
    def __init__(self, d_model, num_heads, forward_expansion, dropout=0.1):  
        super(DecoderStack, self).__init__()
        self.Masked_MHA = MultiHeadAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
        self.Crossed_MHA = MultiHeadAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
        self.FFN = FeedForwardNet(d_model=d_model, forward_expansion=forward_expansion, dropout=dropout)
        self.LayerNorm1 = nn.LayerNorm(d_model)
        self.LayerNorm2 = nn.LayerNorm(d_model)
        self.LayerNorm3 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout)  
        self.dropout2 = nn.Dropout(dropout)  
        self.dropout3 = nn.Dropout(dropout)  

    def forward(self, x, encoder_out, trg_mask):
        masked_att_out = self.dropout1(self.Masked_MHA(x, x, x, trg_mask)) 
        masked_att_out = self.LayerNorm1(masked_att_out + x)
        crossed_att_out = self.dropout2(self.Crossed_MHA(encoder_out, masked_att_out, encoder_out)) 
        crossed_att_out = self.LayerNorm2(crossed_att_out + masked_att_out)
        ffn_out = self.dropout3(self.FFN(crossed_att_out))  
        ffn_out = self.LayerNorm3(ffn_out + crossed_att_out)
        return ffn_out


 #### 3.8. Decoder

 Bu bölümde Decoder bloğunu oluşturuyoruz. Decoder bloğu bir Embedding katmanı ve N adet Decoder Stack bloğundan (N = num_layers) oluşur.

In [ ]:
class Decoder(nn.Module):
    def __init__(self,vocab_size, block_size, d_model, num_heads, forward_expansion, num_layers):
        super().__init__()
        self.block_size = block_size
        self.d_model = d_model
        self.embeding_layer = EmbeddingLayer(d_model, vocab_size, block_size)
        self.layers = nn.ModuleList([DecoderStack(d_model, num_heads, forward_expansion) for _ in range(num_layers)])

    def forward(self, x, encoder_output, trg_mask):
        x = self.embeding_layer(x)
        for layer in self.layers:
            x = layer(x, encoder_output, trg_mask)
        return x


 #### 3.9. Transformer

 Son olarak, Transformer modelini oluşturuyoruz. Transformer modeli, Encoder ve Decoder bloklarının birleşiminden oluşur. Encoder, Embedding katmanı ve N adet Encoder Stack katmanından oluşur. Decoder ise Embedding katmanını ve N adet Decoder Stack katmanından oluşur. En son da bu modelin ucuna çıktı boyutu Tokenizer'ın öğrendiği kelime sayısına eşit bir lineer katman koyarız. Bu son katman sayesinde Decoder'dan aldığımız çıktıyı tahmin yapabilmek için kullanılabilecek hale getirebilriz.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, block_size, d_model, nhead, num_encoder_layers, num_decoder_layers,
                 forward_expansion, learning_rate, dropout=0.1): 
        super(Transformer, self).__init__()
        self.encoder = Encoder(vocab_size, block_size, d_model, nhead, forward_expansion, num_encoder_layers)
        self.decoder = Decoder(vocab_size, block_size, d_model, nhead, forward_expansion, num_decoder_layers)
        
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(src)
        decoder_output = self.decoder(trg, encoder_output, trg_mask)
        output = self.linear(decoder_output)
        return output


## Son

### Herhangi bir sorunuz olursa, bana aşağıdaki iletişim bilgilerimden kolayca ulaşabilirsiniz.

Mail: i_konak@hotmail.com

Linkedin: https://www.linkedin.com/in/ismail-konak/

GitHub: https://github.com/IsmailKonak
